In [ ]:
# Falcon specification: https://falcon-sign.info/falcon.pdf
# Falcon Python implementation: https://github.com/tprest/falcon.py
# NTRU Solve: https://tprest.github.io/pdf/slides/ntru-gen-pkc.pdf

In [4]:
# global constants
n = 2^9
q=12289
sigma_min = 1.277833697 # see table 3.3 page 51
sigma_max = 1.8205
sigma = 165.736617183
beta = sqrt(34034726)

In [6]:
# use variables consistently
# to avoid clash of types, use variables x, X, y, Y, z, Z as described below
Zx.<x> = PolynomialRing(ZZ) 
#Zphi.<X> = Zx.quotient(x^4+1)
#Qy.<y> = PolynomialRing(QQ)
#Qphi.<Y> = Qy.quotient(y^4+1)
#Zq.<z> = PolynomialRing(Integers(7))
#ZQphi.<Z> = Zq.quotient(z^4+1)

In [10]:
# ============================ POLYNOMIAL UTILS ===================================

# algorythm 16 page 45, author: Maxim Pushkar
def Balance(f,q,n):
    g = list(((f[i] + q//2) % q) - q//2 for i in range(n))
    # return f.parent()(g)
    return Zx(g)

# formula 3.21 from page 28
def Split(f,n):
    pass

# formula 3.22 from page 28, author: Evgen Postulga
def Merge(a,b):
    a = a.subs(x=x^2)
    b = x*b.subs(x=x^2)
    return a+b

# formula 3.6 from page 23, author: Evgen Postulga
def HermitianAdjointPoly(p, n):
    f=[p[0]]
    for i in range(1,n):    
        f.append(-p[n-i])
    # return p.parent()(f)
    return Zx(f) 

# formula 3.10 from page 24, author: Evgen Postulga
def InnerProduct(a,b,n):
    # return sum([a[i] * b[i] for i in range(n)])
    s=0
    for i in range(n):
        s=s+a[i]*b[i]
    return(s)

# formula 3.9 from page 24, author: Evgen Postulga
def EuclideanNorm(a,n):
    # return sqrt(InnerProduct(a,a,n))
    b=InnerProduct(a,a,n)
    return b^(1/2)

# formula 3.25 from page 30, author: Maxim Pushkar
def FieldNorm(f, n):
    f0, f1 = split(f, n)
    iks = f.parent()([0, 1])
    return (f0^2 - iks * f1^2) % (iks^(n/2)+1)
    # return (f0^2-x*f1^2)%(x^(n/2)+1)

# NNT representation of polynomial f from Zq[x]/x^n+1
def NTT(f,n,q):
    pass

In [16]:
# ======================== Lattice Matrices ============================

# author: 
def CyclicRotate(input, n):
    pass

# author: 
def PolyToCirculant(p, n):
    pass

# author: Evgen Postulga
def CirculantToPoly(M):
    # Zx.<x> = PolynomialRing(ZZ)
    return Zx(list(M[0]))
    
# author: 
def PolyToLattice4(p00, p01, p10, p11, n):
   pass

# author: Evgen Postulga
def LatticeToPoly4(M, n_2):
    p00=Zx(M[0][:n])
    p01=Zx(M[0][n:])
    p10=Zx(M[n][:n])
    p11=Zx(M[n][n:])
    return p00, p01, p10, p11

# formula 3.7 page 23б author: Evgen Postulga
def HermitianAdjointMatrix(M, n2):
    a, c, b, d = LatticeToPoly4(M, n2/2)
    a, c, b, d = HermitianAdjointPoly(a, n2/2), HermitianAdjointPoly(c, n2/2),HermitianAdjointPoly(b, n2/2),HermitianAdjointPoly(d, n2/2)
    return PolyToLattice4(a, c, b, d, n2/2)

In [ ]:
# =========================== RANDOM GENERATORS ============================

# formula 3.32 on page 40, author: Aleksey Mokhonko
def UniformBits(k):
    return bytes(list(floor(uniform(0, 256)) for i in range(k / 8)))

# algorythm 12 page 41, author: Dmytro Husan
def BaseSampler():
    u = UniformBits(72)
    z_0 = 0
    RCDT = [3024686241123004913666, 1564742784480091954050, 636254429462080897535, 199560484645026482916, 47667343854657281903, 8595902006365044063, 1163297957344668388, 117656387352093658, 8867391802663976, 496969357462633, 20680885154299, 638331848991, 14602316184, 247426747, 3104126, 28824, 198, 1, 0]
    for i in range(0, 18):
        z_0 = z_0 + int(u<RCDT[i]) 
    return z_0

# algorythm 13 page 42, author: Dmytro Husan
def ApproxExp(x, ccs):
    C = (0x00000004741183A3, 0x00000036548CFC06, 0x0000024FDCBF140A, 0x0000171D939DE045,
         0x0000D00CF58F6F84, 0x000680681CF796E3, 0x002D82D8305B0FEA, 0x011111110E066FD0,
         0x0555555555070F00, 0x155555555581FF00, 0x400000000002B400, 0x7FFFFFFFFFFF4800, 0x8000000000000000)
    y = C[0]
    z = floor(2^63*x)
    for i in range(1, 13):
        y = C[i] - (z*y) >> 63
    z = floor(2^63*ccs)
    y = (z*y) >> 63
    return y
    
# algorithm 14 page 43, author: Dmytro Husan
def BerExp(x, ccs):
    s = floor(x/ln(2))
    r = x - s*ln(2)
    s = min(s, 63)
    z = (2*ApproxExp(r, ccs) - 1) >> s
    for i in range(56, -8, -8):
        p = int.from_bytes(urandom(3), "little")
        w = p - ((z >> i) & 0xFF)
        if int(w) == 0:
            break
    return int(w < 0)

# algorithm 15, page 43, author: Dmytro Husan
def SamplerZ(mu, sigma, sigma_min, sigma_max):
    r = mu - floor(mu)
    ccs = sigmamin/sigma
    while True:
        z_0 = BaseSampler()
        b = UniformBits(8)&0x1
        z = b + (2*b-1)*z_0
        x = (z-r)^2/2/sigma^2 - z_0^2/2/sigmamax^2
        if BerExp(x, ccs) == 1:
            return z + floor(mu)